In [ ]:
import dill
dill.load_session('lqh_p3_session.db')

In [1]:
import pandas as pd 
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error


In [14]:
path = 'C:/Users/isaac/Dropbox/Apps/ShareLaTeX/information_lawyer_quality'

# Read data 
data = pd.read_csv(path + '/_aux/betas_score_p3.csv') 
#data = data.sample(frac=0.1, replace=True, random_state=1)
print(data.shape)
data.columns

(3718, 103)


Index(['junta', 'exp', 'anio', 'id', 'gen', 'horas_sem', 'salario_diario',
       'abogado_pub', 'c_antiguedad', 'trabajador_base',
       ...
       'beta_monto', 'se_total', 'se_calif_rubro_proemio',
       'se_calif_prestaciones', 'se_calif_hechos', 'se_calif_derechos',
       'se_calif_puntos_petitorios', 'se_prediccion_a', 'se_prediccion_b',
       'se_monto'],
      dtype='object', length=103)

In [15]:
data.describe()

,junta,exp,anio,id,gen,horas_sem,salario_diario,abogado_pub,c_antiguedad,trabajador_base,...,beta_monto,se_total,se_calif_rubro_proemio,se_calif_prestaciones,se_calif_hechos,se_calif_derechos,se_calif_puntos_petitorios,se_prediccion_a,se_prediccion_b,se_monto
count,3718.000000,3718.000000,3718.000000,3392.000000,3399.000000,3359.000000,3.376000e+03,3393.000000,3217.000000,3401.000000,...,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000
mean,10.313609,1099.723507,2017.890264,21.721993,0.500441,79.271512,7.914960e+03,0.312703,3.667617,0.031755,...,0.037663,0.004769,0.004132,0.015357,0.008919,0.007348,0.012135,0.026624,0.060904,0.152681
std,40.286065,706.215885,0.691473,4.094247,0.500073,1062.669490,7.263470e+04,0.463663,4.465201,0.175374,...,0.347286,0.003438,0.002981,0.010642,0.006555,0.006971,0.009149,0.018686,0.043154,0.109446
min,1.000000,1.000000,2010.000000,1.000000,0.000000,2.000000,1.738000e+01,0.000000,0.000000,0.000000,...,-0.721416,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,598.250000,2017.000000,21.000000,0.000000,45.000000,2.441800e+02,0.000000,0.893151,0.000000,...,-0.010891,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.000000,992.000000,2018.000000,21.000000,1.000000,54.000000,4.577150e+02,0.000000,2.049315,0.000000,...,-0.010891,0.006535,0.005658,0.022170,0.012435,0.008667,0.018167,0.038302,0.085596,0.213322
75%,12.000000,1529.000000,2018.000000,23.000000,1.000000,60.000000,4.350000e+03,1.000000,4.613698,0.000000,...,0.000000,0.006535,0.005658,0.022170,0.012435,0.008667,0.018167,0.038302,0.085596,0.213322
max,1666.000000,3467.000000,2019.000000,36.000000,1.000000,43598.000000,1.862859e+06,1.000000,33.939728,1.000000,...,5.375523,0.016149,0.015437,0.039964,0.041997,0.042067,0.043980,0.077791,0.184290,0.484216


In [96]:
Xvars = ["gen",
            "horas_sem",
            "salario_diario",
            "abogado_pub",
            "c_antiguedad",
            "trabajador_base",
            "indem",
            "sal_caidos",
            "prima_antig",
            "prima_vac",
            "hextra",
            "prima_dom",
            "desc_sem",
            "desc_ob",
            "sarimssinf",
            "utilidades",
            "nulidad",
            "codem",
            "reinst",
            "giro_empresa",
            "accion_principal",
            "derecho_propio_c_clv",
            "giro_defendant_c_clv",
            "num_demandados_totales",
            "num_demandados_completos",
            "num_abogados_proemio",
            "num_abogados_proemio_completos",
            "incrementos_sal_c_clv",
            "contrato_c_clv",
            "lugar_trabajo_c_clv",
            "horario_inicio_jornada_c_clv",
            "horario_fin_jornada_c_clv",
            "especifican_dias_c_clv",
            "salario_int_clv",
            "periodo_sal_int_clv",
            "fecha_despido_c_clv",
            "lugar_despido_c_clv",
            "motivo_despido_c_clv",
            "cargo_despide_clv",
            "presentado_tiempo_forma_c_clv",
            "personalidad_c_clv",
            "firma_trabajador_c_clv"]
Zvars = ["tipo_persona_demandado",
            "dummy_correccion_domicilio",
            "dummy_ampliacion_modif",
            "dummy_exhorto",
            "contador_notificacion_actor",
            "comparecencia_cde_actor",
            "contador_cde",
            "comparecencia_oap_actor",
            "contador_oap",
            "comparecencia_dp_actor",
            "contador_dp",
            "alegatos_actor",
            "confes_ofr_actor",
            "testi_ofr_actor",
            "docum_ofr_actor",
            "inspec_ofr_actor",
            "peric_ofr_actor",
            "confes_adm_actor",
            "testi_adm_actor",
            "docum_adm_actor",
            "inspec_adm_actor",
            "peric_adm_actor",
            "confes_desahg_actor",
            "testi_desahog_actor",
            "docum_desahog_actor",
            "inspec_desahog_actor",
            "peric_desahog_actor",
            "confes_desahog_desist_actor",
            "testi_desahog_desist_actor",
            "docum_objecion_actor",
            "dummy_inspec_compar_actor",
            "dummy_inspec_mostro_actor"]

X = data[Xvars]
Z = data[Zvars]
W = pd.concat([X,Z], axis=1)

In [97]:
i = 0
y1 = pd.DataFrame()
y2 = pd.DataFrame()

for vdeps in ["total", "calif_rubro_proemio", "calif_prestaciones", "calif_hechos", "calif_derechos", "calif_puntos_petitorios", "monto"] :
    y1[i] = data[vdeps]
    y1.rename(columns={i: vdeps}, inplace=True)
    beta_col = "beta_" + vdeps 
    y2[i] = data[beta_col]
    y2.rename(columns={i: beta_col}, inplace=True)    
    i = i+1
 
    

In [98]:
numeric_featuresX = ["horas_sem",
            "salario_diario", 
            "c_antiguedad", 
            "num_demandados_totales",
            "num_demandados_completos",
            "num_abogados_proemio",
            "num_abogados_proemio_completos"]

numeric_featuresZ = ["tipo_persona_demandado",
            "dummy_correccion_domicilio",
            "dummy_ampliacion_modif",
            "dummy_exhorto",
            "contador_notificacion_actor",
            "comparecencia_cde_actor",
            "contador_cde",
            "comparecencia_oap_actor",
            "contador_oap",
            "comparecencia_dp_actor",
            "contador_dp",
            "alegatos_actor",
            "confes_ofr_actor",
            "testi_ofr_actor",
            "docum_ofr_actor",
            "inspec_ofr_actor",
            "peric_ofr_actor",
            "confes_adm_actor",
            "testi_adm_actor",
            "docum_adm_actor",
            "inspec_adm_actor",
            "peric_adm_actor",
            "confes_desahg_actor",
            "testi_desahog_actor",
            "docum_desahog_actor",
            "inspec_desahog_actor",
            "peric_desahog_actor",
            "confes_desahog_desist_actor",
            "testi_desahog_desist_actor"]

numeric_featuresW = numeric_featuresX + numeric_featuresZ

In [99]:
categorical_featuresX = list(set(Xvars) - set(numeric_featuresX))
categorical_featuresZ = list(set(Zvars) - set(numeric_featuresZ))
categorical_featuresW = categorical_featuresX + categorical_featuresZ

In [100]:

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler())])


categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse = False)


preprocessorX = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresX),
        ('cat', categorical_transformer, categorical_featuresX)])
preprocessorZ = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresZ),
        ('cat', categorical_transformer, categorical_featuresZ)])
preprocessorW = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_featuresW),
        ('cat', categorical_transformer, categorical_featuresW)])

In [101]:
Xy = pd.concat([W, y2['beta_total']], axis=1)
Xy = Xy.dropna()

In [102]:
F = Xy.drop('beta_total', axis=1)
y = Xy['beta_total']

In [103]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(F, y, test_size=0.2,
                                                    random_state=0)

In [104]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
from sklearn.linear_model import LinearRegression

In [107]:
# Linear Regression
LRX = Pipeline(steps=[('preprocessor', preprocessorW),
                      ('model', LinearRegression())])
LRX.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['horas_sem',
                                                   'salario_diario',
                                                   'c_antiguedad',
                                                   'num_demandados_totales',
                                                   'num_demandados_completos',
                    

In [108]:
from sklearn.ensemble import GradientBoostingRegressor

In [109]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'model__n_estimators'  : [500, 1000, 2000],
              'model__max_depth'     : [None, 1, 3, 5, 10, 20],
              'model__learning_rate' : [0.001, 0.01, 0.1]
             }


GB  = GradientBoostingRegressor(
                        random_state        = 123,
                        # Activación de la parada temprana
                        validation_fraction = 0.1,
                        n_iter_no_change    = 5,
                        tol                 = 0.0001
                    )
# Gradient Boosting
GBX = Pipeline(steps=[('preprocessor', preprocessorW),
                      ('model', GB)])

# Búsqueda por grid search con validación cruzada
# ==============================================================================
gridX = GridSearchCV(GBX,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )

gridX.fit(X = X_train, y = y_train)
modelo_final_GBX = gridX.best_estimator_

Fitting 15 folds for each of 54 candidates, totalling 810 fits


In [110]:
from sklearn.ensemble import RandomForestRegressor

In [111]:
param_grid = {'model__n_estimators': [500, 1000, 2000],
              'model__max_depth'   : [None, 3, 10, 20]
             }

# Random Forests
RFX = Pipeline(steps=[('preprocessor', preprocessorW),
                      ('model', RandomForestRegressor(random_state=123))])


# Búsqueda por grid search con validación cruzada
# ==============================================================================
gridX = GridSearchCV(
        estimator  = RFX,
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 4,
        return_train_score = True
       )


gridX.fit(X = X_train, y = y_train)
modelo_final_RFX = gridX.best_estimator_

Fitting 15 folds for each of 12 candidates, totalling 180 fits


In [112]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.cross_decomposition import PLSRegression
#from sklearn.pipeline import make_pipeline

In [113]:
# PCR
pcrX = Pipeline(steps=[('preprocessor', preprocessorW),
                     ('components', TruncatedSVD(n_components=10)),
                      ('model', LinearRegression())])


pcrX.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['horas_sem',
                                                   'salario_diario',
                                                   'c_antiguedad',
                                                   'num_demandados_totales',
                                                   'num_demandados_completos',
                    

In [114]:
from sklearn.preprocessing import FunctionTransformer

plsX = Pipeline(steps=[('preprocessor', preprocessorW),
                    ('model', PLSRegression(n_components=10))])



In [115]:
plsX.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('polynomial',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['horas_sem',
                                                   'salario_diario',
                                                   'c_antiguedad',
                                                   'num_demandados_totales',
                                                   'num_demandados_completos',
                    

In [116]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from scipy.stats import uniform

In [117]:
# Espacio de búsqueda de cada hiperparámetro
# ==============================================================================
param_distributions = {
    'modelo__alpha': uniform(0.001, 10) ,
    'modelo__l1_ratio' : uniform(0.019, 0.99)
}

# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
ENX = Pipeline([('preprocessing', preprocessorW),
                 ('modelo', ElasticNet(random_state=123))])


# Búsqueda por validación cruzada
# ==============================================================================
gridX = RandomizedSearchCV(
        estimator  = ENX,
        param_distributions = param_distributions,
        n_iter     = 10,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )


gridX.fit(X = X_train, y = y_train)
modelo_final_ENX = gridX.best_estimator_

Fitting 15 folds for each of 10 candidates, totalling 150 fits


In [118]:
# Espacio de búsqueda de cada hiperparámetro
# ==============================================================================
param_distributions = {
    'modelo__alpha': uniform(0.001, 10)
}

# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
LASSOX = Pipeline([('preprocessing', preprocessorW),
                 ('modelo', Lasso(random_state=123))])



# Búsqueda por validación cruzada
# ==============================================================================
gridX = RandomizedSearchCV(
        estimator  = LASSOX,
        param_distributions = param_distributions,
        n_iter     = 10,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )


gridX.fit(X = X_train, y = y_train)
modelo_final_LASSOX = gridX.best_estimator_

Fitting 15 folds for each of 10 candidates, totalling 150 fits


In [119]:
# Se combinan los pasos de preprocesado y el modelo en un mismo pipeline
RIDGEX = Pipeline([('preprocessing', preprocessorW),
                 ('modelo', Ridge())])




# Búsqueda por validación cruzada
# ==============================================================================
gridX = RandomizedSearchCV(
        estimator  = RIDGEX,
        param_distributions = param_distributions,
        n_iter     = 10,
        scoring    = 'neg_mean_squared_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123),  
        verbose    = 4,
        random_state = 123,
        return_train_score = True
       )

gridX.fit(X = X_train, y = y_train)
modelo_final_RIDGEX = gridX.best_estimator_


Fitting 15 folds for each of 10 candidates, totalling 150 fits


In [120]:
import dill
dill.dump_session('lqh_session_p3.db')

In [121]:
#Metric Results

metric_results_oosX = pd.DataFrame({'Linear Regression' :  [explained_variance_score(y_test, LRX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, LRX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, LRX.predict(X_test)),
    mean_squared_error(y_test, LRX.predict(X_test)),
    mean_absolute_percentage_error(y_test, LRX.predict(X_test))], 
                                     'GB' :  [explained_variance_score(y_test, modelo_final_GBX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_GBX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_GBX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_GBX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_GBX.predict(X_test))],
                                    'RF' :  [explained_variance_score(y_test, modelo_final_RFX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_RFX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_RFX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_RFX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_RFX.predict(X_test))],
                                    'PC Regression' :  [explained_variance_score(y_test, pcrX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, pcrX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, pcrX.predict(X_test)),
    mean_squared_error(y_test, pcrX.predict(X_test)),
    mean_absolute_percentage_error(y_test, pcrX.predict(X_test))], 
                                    'PLS' :  [explained_variance_score(y_test, plsX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, plsX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, plsX.predict(X_test)),
    mean_squared_error(y_test, plsX.predict(X_test)),
    mean_absolute_percentage_error(y_test, plsX.predict(X_test))], 
                                     'ElasticNet' :  [explained_variance_score(y_test, modelo_final_ENX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_ENX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_ENX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_ENX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_ENX.predict(X_test))],
                                     'LASSO' :  [explained_variance_score(y_test, modelo_final_LASSOX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_LASSOX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_LASSOX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_LASSOX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_LASSOX.predict(X_test))],
                                     'RIDGE' :  [explained_variance_score(y_test, modelo_final_RIDGEX.predict(X_test), multioutput='uniform_average'),
    r2_score(y_test, modelo_final_RIDGEX.predict(X_test), multioutput='variance_weighted'),
    mean_absolute_error(y_test, modelo_final_RIDGEX.predict(X_test)),
    mean_squared_error(y_test, modelo_final_RIDGEX.predict(X_test)),
    mean_absolute_percentage_error(y_test, modelo_final_RIDGEX.predict(X_test))]                                 
                        })    

In [122]:
metric_results_insX = pd.DataFrame({'Linear Regression' :  [explained_variance_score(y_train, LRX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, LRX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, LRX.predict(X_train)),
    mean_squared_error(y_train, LRX.predict(X_train)),
    mean_absolute_percentage_error(y_train, LRX.predict(X_train))], 
                                     'GB' :  [explained_variance_score(y_train, modelo_final_GBX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_GBX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_GBX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_GBX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_GBX.predict(X_train))],
                                    'RF' :  [explained_variance_score(y_train, modelo_final_RFX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_RFX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_RFX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_RFX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_RFX.predict(X_train))],
                                    'PC Regression' :  [explained_variance_score(y_train, pcrX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, pcrX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, pcrX.predict(X_train)),
    mean_squared_error(y_train, pcrX.predict(X_train)),
    mean_absolute_percentage_error(y_train, pcrX.predict(X_train))], 
                                    'PLS' :  [explained_variance_score(y_train, plsX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, plsX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, plsX.predict(X_train)),
    mean_squared_error(y_train, plsX.predict(X_train)),
    mean_absolute_percentage_error(y_train, plsX.predict(X_train))], 
                                     'ElasticNet' :  [explained_variance_score(y_train, modelo_final_ENX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_ENX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_ENX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_ENX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_ENX.predict(X_train))],
                                     'LASSO' :  [explained_variance_score(y_train, modelo_final_LASSOX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_LASSOX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_LASSOX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_LASSOX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_LASSOX.predict(X_train))],
                                     'RIDGE' :  [explained_variance_score(y_train, modelo_final_RIDGEX.predict(X_train), multioutput='uniform_average'),
    r2_score(y_train, modelo_final_RIDGEX.predict(X_train), multioutput='variance_weighted'),
    mean_absolute_error(y_train, modelo_final_RIDGEX.predict(X_train)),
    mean_squared_error(y_train, modelo_final_RIDGEX.predict(X_train)),
    mean_absolute_percentage_error(y_train, modelo_final_RIDGEX.predict(X_train))]                                       
                        })     
 

In [123]:
# Change the row indexes
metric_results_oosX.index = ['Explained Variance', 'R2', 'MAE', 'MSE', 'MAPE']
metric_results_insX.index = ['Explained Variance', 'R2', 'MAE', 'MSE', 'MAPE']

In [68]:
metric_results_oosW

,Linear Regression,GB,RF,PC Regression,PLS,ElasticNet,LASSO,RIDGE
Explained Variance,-7.789229e+22,3.185681e-01,3.237152e-01,2.663905e-02,1.284529e-01,0.000000e+00,0.000000e+00,6.722775e-02
R2,-7.800159e+22,3.150974e-01,3.194800e-01,2.406994e-02,1.207656e-01,-2.829618e-03,-2.829618e-03,6.319888e-02
MAE,1.621483e+09,3.481603e-02,3.099247e-02,4.450057e-02,3.647744e-02,4.689456e-02,4.689456e-02,3.595189e-02
MSE,2.049550e+20,1.799632e-03,1.788117e-03,2.564329e-03,2.310254e-03,2.635009e-03,2.635009e-03,2.461514e-03
MAPE,4.257658e+24,6.216656e+13,5.670796e+13,7.956754e+13,6.545928e+13,8.558003e+13,8.558003e+13,6.226150e+13


In [69]:
metric_results_insW

,Linear Regression,GB,RF,PC Regression,PLS,ElasticNet,LASSO,RIDGE
Explained Variance,5.568651e-01,5.563999e-01,6.247906e-01,9.611366e-02,3.943742e-01,0.000000e+00,0.000000e+00,4.896029e-01
R2,5.568641e-01,5.563931e-01,6.247748e-01,9.611366e-02,3.943742e-01,0.000000e+00,0.000000e+00,4.896029e-01
MAE,2.364546e-02,2.808349e-02,2.162073e-02,4.190818e-02,2.993488e-02,4.615278e-02,4.615278e-02,2.629788e-02
MSE,1.105752e-03,1.106927e-03,9.362953e-04,2.255458e-03,1.511211e-03,2.495289e-03,2.495289e-03,1.273588e-03
MAPE,4.057472e+13,5.013114e+13,3.794436e+13,7.420535e+13,5.059923e+13,8.344897e+13,8.344897e+13,4.655775e+13


In [91]:
metric_results_oosZ

,Linear Regression,GB,RF,PC Regression,PLS,ElasticNet,LASSO,RIDGE
Explained Variance,-6.522599e+23,1.089227e-01,1.284551e-01,1.527570e-02,3.976553e-02,0.000000e+00,0.000000e+00,6.805648e-02
R2,-6.522744e+23,1.055684e-01,1.231464e-01,1.225291e-02,3.535465e-02,-4.137597e-03,-4.137597e-03,6.495008e-02
MAE,3.312595e+09,3.846388e-02,3.704176e-02,4.269777e-02,3.940010e-02,4.361078e-02,4.361078e-02,3.809683e-02
MSE,1.517556e+21,2.080950e-03,2.040054e-03,2.298054e-03,2.244307e-03,2.336188e-03,2.336188e-03,2.175451e-03
MAPE,4.564994e+24,7.057292e+13,6.743122e+13,7.742809e+13,7.303185e+13,7.866046e+13,7.866046e+13,7.240554e+13


In [92]:
metric_results_insZ

,Linear Regression,GB,RF,PC Regression,PLS,ElasticNet,LASSO,RIDGE
Explained Variance,8.701909e-02,2.815006e-01,3.150808e-01,4.461808e-02,1.623701e-01,2.220446e-16,2.220446e-16,2.347409e-01
R2,8.696352e-02,2.813749e-01,3.150693e-01,4.461808e-02,1.623701e-01,0.000000e+00,0.000000e+00,2.347409e-01
MAE,3.841232e-02,3.506522e-02,3.295633e-02,4.218895e-02,3.762637e-02,4.372943e-02,4.372943e-02,3.504034e-02
MSE,2.118225e-03,1.667195e-03,1.589025e-03,2.216466e-03,1.943284e-03,2.319979e-03,2.319979e-03,1.775385e-03
MAPE,6.964519e+13,6.539570e+13,6.241405e+13,7.686083e+13,6.957024e+13,7.791557e+13,7.791557e+13,6.574500e+13


In [124]:
metric_results_oosX

,Linear Regression,GB,RF,PC Regression,PLS,ElasticNet,LASSO,RIDGE
Explained Variance,-2.746579e+21,9.468609e-01,0.950408,2.149241e-01,8.069774e-01,0.000000e+00,0.000000e+00,8.790763e-01
R2,-2.751172e+21,9.467993e-01,0.950394,2.148452e-01,8.059393e-01,-9.110451e-04,-9.110451e-04,8.789571e-01
MAE,2.620498e+08,5.840553e-03,0.003116,3.750452e-02,1.443382e-02,4.591897e-02,4.591897e-02,1.022335e-02
MSE,6.777230e+18,1.310545e-04,0.000122,1.934148e-03,4.780486e-04,2.465642e-03,2.465642e-03,2.981768e-04
MAPE,1.056281e+13,7.195280e+12,0.081357,6.053760e+13,1.300278e+13,8.001786e+13,8.001786e+13,9.565573e+12


In [125]:
metric_results_insX

,Linear Regression,GB,RF,PC Regression,PLS,ElasticNet,LASSO,RIDGE
Explained Variance,9.391869e-01,9.842852e-01,0.996938,2.316097e-01,8.376219e-01,0.000000e+00,0.000000e+00,9.091762e-01
R2,9.391865e-01,9.842761e-01,0.996933,2.316097e-01,8.376219e-01,0.000000e+00,0.000000e+00,9.091762e-01
MAE,7.425596e-03,4.855851e-03,0.000933,3.699665e-02,1.306845e-02,4.686228e-02,4.686228e-02,8.684831e-03
MSE,1.610749e-04,4.164735e-05,0.000008,2.035212e-03,4.300857e-04,2.648669e-03,2.648669e-03,2.405621e-04
MAPE,1.008904e+13,7.662590e+12,0.023837,6.135356e+13,1.339017e+13,8.521476e+13,8.521476e+13,9.288088e+12
